## 4.5.2 Processing of census data

The purpose of this notebook is to clean and preprocess the data to handle missing values, standardize formats, and correct any inconsistencies.


### Import libraries and define paths

In [1]:
import pandas as pd
import os
import ast
import re
import numpy as np
from pandas.errors import SettingWithCopyWarning
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
from math import cos, asin, sqrt
from functools import reduce

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
raw = '../../../../data/3_external_data/census/RAW'
conf_table = pd.read_csv("../CONF/conf_ages.csv")

### Loading tables 

In [4]:
cs_municipio = pd.read_csv(f'{raw}/mapeo_cs_municipio.csv', 
                           dtype={'Seccion censal':str, 'Municipio':str, 'cpro':str, 'cmun':str, 'dist':str, 'secc':str})

In [5]:
df_sexo_edad_sc = pd.read_csv(f'{raw}/raw_pop_gen_age_cs.csv', converters={'Data': ast.literal_eval})
df_sexo_nacionalidad_sc = pd.read_csv(f'{raw}/raw_pop_gen_nat_cs.csv', converters={'Data': ast.literal_eval})
df_renta_media = pd.read_csv(f'{raw}/mean_median_rent.csv',converters={'Data': ast.literal_eval})
df_indicadores_leyenda = pd.read_excel(f'{raw}/legend_indicators.xlsx', header=7, sheet_name='indicadores')
df_datos_indicadores = pd.read_excel(f'{raw}/indicators.xlsx', dtype={'cpro': str, 'cmun' : str , 'dist' : str, 'secc':str})

### Transforming Tables

In [6]:
df_sexo_edad_sc[['Sexo', 'Seccion censal', 'Rango edad']] = df_sexo_edad_sc['Nombre'].str.split(',', expand=True)
df_sexo_edad_sc[['Poblacion', 'Secreto']] = pd.json_normalize(df_sexo_edad_sc['Data'].apply(lambda x: x[0]))
df_sexo_edad_sc.drop(['Nombre', 'Data', 'Secreto'], axis = 1, inplace = True)
df_sexo_edad_sc['Poblacion'] = df_sexo_edad_sc['Poblacion'].astype(int)
df_sexo_edad_sc = df_sexo_edad_sc.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_sexo_edad_sc = df_sexo_edad_sc[~(df_sexo_edad_sc['Seccion censal'] == 'TOTAL')].reset_index(drop=True)
df_sexo_edad_sc['Sexo'] = df_sexo_edad_sc['Sexo'].replace('Mujeres', 'Mujer').replace('Hombres', 'Hombre')
df_sexo_edad_sc

,Sexo,Seccion censal,Rango edad,Poblacion
0,Ambos Sexos,0100101001,Total,1388
1,Ambos Sexos,0100101001,0-4,45
2,Ambos Sexos,0100101001,5-9,71
3,Ambos Sexos,0100101001,10-14,118
4,Ambos Sexos,0100101001,15-19,94
...,...,...,...,...
2400547,Mujer,5200108015,80-84,17
2400548,Mujer,5200108015,85-89,13
2400549,Mujer,5200108015,90-94,6
2400550,Mujer,5200108015,95-99,4


In [7]:
df_sexo_edad_sc = pd.merge(df_sexo_edad_sc, age_mapping, left_on = 'Rango edad', right_on = 'Edad INE', how='left').drop(['Edad movilidad', 'Rango edad', 'Edad INE'], axis = 1)
df_sexo_edad_sc = df_sexo_edad_sc.groupby(['Sexo', 'Seccion censal', 'Edad tecnologías']).sum().reset_index()
df_sexo_edad_sc.rename(columns = {'Edad tecnologías': 'Edad'}, inplace=True)

In [8]:
df_sexo_edad_sc.rename(columns={'Sexo': 'Gender',
                                'Seccion censal': 'Census section',
                                'Edad': 'Age',
                                'Poblacion': 'Population'
                               }, 
                       inplace=True)

In [9]:
df_sexo_edad_sc

,Gender,Census section,Age,Population
0,Ambos Sexos,0100101001,De 30 a 39 años,135
1,Ambos Sexos,0100101001,De 40 a 49 años,304
2,Ambos Sexos,0100101001,De 50 a 59 años,247
3,Ambos Sexos,0100101001,De 60 a 69 años,133
4,Ambos Sexos,0100101001,De 70 y más años,119
...,...,...,...,...
763807,Mujer,5200108015,De 50 a 59 años,197
763808,Mujer,5200108015,De 60 a 69 años,139
763809,Mujer,5200108015,De 70 y más años,100
763810,Mujer,5200108015,Menos de 30 años,407


### Transforms 2

In [10]:
# We separate the "Nombre" column into "Sexo", "Seccion censal", and "Nacionalidad". We do the same with the "Data" column where we are only interested in "Valor"
df_sexo_nacionalidad_sc[['Sexo', 'Seccion censal', 'Nacionalidad']] = df_sexo_nacionalidad_sc['Nombre'].str.split(',', expand=True)
df_sexo_nacionalidad_sc[['Valor', 'Secreto']] = pd.json_normalize(df_sexo_nacionalidad_sc['Data'].apply(lambda x: x[0]))
# We remove the columns that we have already used and cast the "Valor" column to int
df_sexo_nacionalidad_sc.drop(['Nombre', 'Data', 'Secreto'], axis = 1, inplace = True)
df_sexo_nacionalidad_sc['Valor'] = df_sexo_nacionalidad_sc['Valor'].astype(int)
# We remove the spaces before and after the strings (There were some cases where ' TOTAL' or ' 0-5' appeared)
df_sexo_nacionalidad_sc = df_sexo_nacionalidad_sc.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# We remove the rows that have the value TOTAL in "seccion censal", as we are not interested in that grouping
df_sexo_nacionalidad_sc = df_sexo_nacionalidad_sc[~(df_sexo_nacionalidad_sc['Seccion censal'] == 'TOTAL')].reset_index(drop=True)
df_sexo_nacionalidad_sc = df_sexo_nacionalidad_sc.pivot_table(df_sexo_nacionalidad_sc, index=['Seccion censal', 'Sexo'], columns= 'Nacionalidad').reset_index(level=[0,1])
df_sexo_nacionalidad_sc.columns = [c[0] if c[1] == '' else c[1] for c in df_sexo_nacionalidad_sc.columns]
df_sexo_nacionalidad_sc['Sexo'] = df_sexo_nacionalidad_sc['Sexo'].replace('Mujeres', 'Mujer').replace('Hombres', 'Hombre')

In [11]:
df_sexo_nacionalidad_sc.rename(columns={'Seccion censal': 'Census section',
                                        'Sexo': 'Gender',
                                        'Africana': 'African',
                                        'Americana': 'American',
                                        'Asiática': 'Asian',
                                        'Española': 'Spanish',
                                        'Europea (excepto Española)': 'European (except Spanish)',
                                        'Resto': 'Rest',
                                        'Total Extranjera': 'Total Foreigner'                                       
                                       },
                                inplace = True)

In [12]:
df_sexo_nacionalidad_sc

,Census section,Gender,African,American,Asian,Spanish,European (except Spanish),Rest,Total,Total Foreigner
0,0100101001,Ambos Sexos,49,21,3,1277,35,3,1388,111
1,0100101001,Hombre,32,6,3,658,27,1,727,69
2,0100101001,Mujer,17,15,0,619,8,2,661,42
3,0100101002,Ambos Sexos,122,35,4,1373,30,8,1572,199
4,0100101002,Hombre,58,18,1,704,18,4,803,99
...,...,...,...,...,...,...,...,...,...,...
109111,5200108014,Hombre,46,2,0,546,1,0,595,49
109112,5200108014,Mujer,43,2,0,569,1,0,615,46
109113,5200108015,Ambos Sexos,237,29,1,2104,17,0,2388,284
109114,5200108015,Hombre,126,17,1,1071,11,0,1226,155


### Transform 3 

In [13]:
# We take the keys of the dictionaries from Data, which will be columns in our tables
claves = list(df_renta_media.Data[0][0].keys())
# We separate the "Nombre" column into "Sexo", "Seccion censal", and "Rango edad". We do the same with the "Data" column where we are only interested in "Valor"
df_renta_media[['Municipio', 'Borra', 'Tipo Dato']] = df_renta_media['Nombre'].str.split('.', expand=True)[[0,1,2]]
df_renta_media['Municipio'] = np.where(df_renta_media['Municipio'] == 'Dato base', df_renta_media['Tipo Dato'], df_renta_media['Municipio'])
df_renta_media['Tipo Dato'] = np.where(df_renta_media['Tipo Dato'].str.contains('sección'), df_renta_media['Borra'], df_renta_media['Tipo Dato'])
# We filter the data to have only values from the latest available year:

df_renta_media['Data'] = df_renta_media['Data'].apply(lambda x : list(filter(lambda d: d.get('Anyo') == max(df_renta_media['Data'].iloc[0], key=lambda x:x['Anyo'])['Anyo'] , x)))
# We extract the records that do not have values in 2020 and remove them
empty_lists = df_renta_media[df_renta_media['Data'].apply(lambda x: len(x) == 0)].index
df_renta_media = df_renta_media.drop(empty_lists)
# We separate the "Data" column into different columns to get the "Valor"
df_renta_media['Valor'] = df_renta_media['Data'].apply(lambda x: x[0]['Valor'])
# Now that we have a table with 2020 data, we take only the data that interests us
df_renta_media2 = df_renta_media[['Municipio','Tipo Dato','Valor']]
# We filter the records that have the word 'sección' in 'Municipio'
df_renta_media2 = df_renta_media2.loc[df_renta_media2['Municipio'].str.contains('sección')]
df_renta_media2.head()
# We convert the values to integers:
df_renta_media2.loc[:, 'Valor'] = df_renta_media2['Valor'].fillna(0)
df_renta_media2['Valor'] = df_renta_media2['Valor'].astype('int64')
# df2.loc[:, 'Valor'] = df2['Valor'].replace(-1, np.nan)
# We separate the municipalities from the municipality codes using the word 'sección'
df_renta_media2[['Municipio', 'Seccion_Municipio']] = df_renta_media2['Municipio'].str.split(' sección ', expand=True)
df_renta_media2['Municipio'] = df_renta_media2['Municipio'].str.strip()
df_renta_media2.head()

,Municipio,Tipo Dato,Valor,Seccion_Municipio
12,Abengibre,Renta neta media por persona,12188,01001
13,Abengibre,Renta neta media por hogar,29768,01001
14,Abengibre,Media de la renta por unidad de consumo,18067,01001
15,Abengibre,Mediana de la renta por unidad de consumo,16450,01001
16,Abengibre,Renta bruta media por persona,14017,01001


In [15]:
cs_municipio['dist_secc'] = cs_municipio['dist'] + cs_municipio['secc']
cs_municipio = cs_municipio.rename(columns={"Municipio": "CodMunicipio"})
# We perform a left join of 'df_renta_media2' with the 'seccion censal' column from 'mapeo_codigos'
tabla_final = df_renta_media2.merge(cs_municipio[['Nombre_municipio','dist_secc','Seccion censal']], how = 'left', 
                                    left_on = ['Municipio', 'Seccion_Municipio'], right_on = ['Nombre_municipio','dist_secc'])

In [16]:
tabla_final = tabla_final.groupby(['Seccion censal','Municipio', 'Tipo Dato','Seccion_Municipio']).sum().reset_index()
# We pivot to have the indicator values in the columns
t_final_renta = tabla_final.pivot(index = ['Municipio','Seccion censal','Seccion_Municipio'], columns = 'Tipo Dato', values = 'Valor').rename_axis(columns=None).reset_index()
t_final_renta.drop(['Municipio', 'Seccion_Municipio'], axis=1, inplace=True)
t_final_renta.columns = t_final_renta.columns.str.strip()

/var/folders/1n/39qxy13x5pv09l_kd3x_11gc0000gn/T/ipykernel_3833/3345904758.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tabla_final = tabla_final.groupby(['Seccion censal','Municipio', 'Tipo Dato','Seccion_Municipio']).sum().reset_index()


In [17]:
translations = {
    'Seccion censal': 'Census section',
    'Media de la renta por unidad de consumo': 'Average income per consumption unit',
    'Mediana de la renta por unidad de consumo': 'Median income per consumption unit',
    'Renta bruta media por hogar': 'Average gross income per household',
    'Renta bruta media por persona': 'Average gross income per person',
    'Renta neta media por hogar': 'Average net income per household',
    'Renta neta media por persona': 'Average net income per person'
}

In [18]:
t_final_renta.rename(columns=translations, inplace=True)

In [19]:
t_final_renta

,Census section,Average income per consumption unit,Median income per consumption unit,Average gross income per household,Average gross income per person,Average net income per household,Average net income per person
0,4400101001,0,0,33059,16846,28059,14299
1,4000101001,19066,17150,37122,14590,31941,12554
2,4800101001,25844,24150,53659,20766,43949,17008
3,4800101002,23689,20650,46269,19431,39387,16540
4,4800101003,19743,19250,36872,15654,32043,13604
...,...,...,...,...,...,...,...
35338,2309204003,14490,12950,27480,11017,24274,9732
35339,2309204004,15411,13650,28589,12123,25098,10643
35340,2309204005,15898,14350,32939,12157,28198,10408
35341,2309205001,13869,12250,25815,10770,22560,9412


### Transform 4 

In [20]:
df_indicadores_leyenda = df_indicadores_leyenda[df_indicadores_leyenda['Indicador'].notna()]
columnas_seccion_censal = ['cpro', 'cmun', 'dist', 'secc']
df_datos_indicadores.insert(0, 'Seccion censal', df_datos_indicadores[columnas_seccion_censal].apply(lambda row: ''.join(row.values.astype(str)), axis=1))
df_datos_indicadores.drop(columnas_seccion_censal, axis = 1, inplace=True)
df_datos_indicadores.drop('ccaa', axis = 1, inplace=True)
df_datos_indicadores.rename(columns=dict(zip(df_indicadores_leyenda["Tabla"], df_indicadores_leyenda["Indicador"])), inplace=True)

In [21]:
translations = {
    'Seccion censal': 'Census section',
    'Total Personas': 'Total people',
    'Porcentaje de mujeres': 'Percentage of women',
    'Porcentaje de hombres': 'Percentage of men',
    'Edad media': 'Average age',
    'Porcentaje de personas menores de 16 años': 'Percentage of people under 16',
    'Porcentaje de personas entre 16 (incluido) y 64 (incluido) años': 'Percentage of people between 16 and 64 years',
    'Porcentaje de personas con más de 64 años': 'Percentage of people over 64 years',
    'Porcentaje extranjeros': 'Percentage of foreigners',
    'Porcentaje personas nacidas en el extranjero': 'Percentage of people born abroad',
    'Porcentaje personas cursando estudios superiores (escur =08 09 10 11 12 ) sobre población de 16 y más': 'Percentage of people pursuing higher education (ages 16 and older)',
    'Porcentaje de personas cursando estudios universitarios (escur = 09 10 11 12 ) sobre población de 16 y más': 'Percentage of university students (ages 16 and older)',
    'Porcentaje personas con estudios superiores  (esreal_cneda=08 09 10 11 12) sobre población de 16 y más': 'Percentage of people with higher education (ages 16 and older)',
    'Porcentaje de población parada sobre población activa= Parados /Activos': 'Unemployment rate among the active population',
    'Porcentaje de población ocupada sobre población de 16 y más =Ocupados/ Pob 16 y +': 'Employment rate among people aged 16 and older',
    'Porcentaje de población activa sobre población de 16 y más= Activos / Pob 16 y +': 'Labor force participation rate among people aged 16 and older',
    'Porcentaje de población pensionista por invalidez sobre población de 16 y más=Pensionistas por invalidez / Pob 16 y +': 'Percentage of disability pensioners among people aged 16 and older',
    'Porcentaje de población pensionista por jubilación sobre población de 16 y más=Pensionistas por jubilación / Pob 16 y +': 'Percentage of retirement pensioners among people aged 16 and older',
    'Porcentaje de población en otra situación de inactividad sobre población de 16 y más=Población en otra situación de inactividad / Pob 16 y +': 'Percentage of people in other inactive statuses among those aged 16 and older',
    'Porcentaje de población estudiante sobre población de 16 y más=Estudiantes / Pob 16 y +': 'Percentage of students among people aged 16 and older',
    'Porcentaje de personas con estado civil soltero': 'Percentage of single individuals',
    'Porcentaje de personas con estado civil casado': 'Percentage of married individuals',
    'Porcentaje de personas personas con estado civil viudo': 'Percentage of widowed individuals',
    'Porcentaje de personas para las que no consta su estado civil ': 'Percentage of people with unknown marital status',
    'Porcentaje de personas con estado civil separado legalmente o divorciado': 'Percentage of legally separated or divorced individuals',
    'Total Viviendas': 'Total housing units',
    'Viviendas Principales': 'Main housing units',
    'Viviendas No principales': 'Secondary housing units',
    'Viviendas en propiedad': 'Owner-occupied housing units',
    'Viviendas en alquiler': 'Rented housing units',
    'Viviendas en otro tipo de régimen de tenencia': 'Housing units under other types of tenure',
    'Total Hogares': 'Total households',
    'Hogares de 1 persona': 'Single-person households',
    'Hogares de 2 personas': 'Two-person households',
    'Hogares de 3 personas': 'Three-person households',
    'Hogares de 4 personas': 'Four-person households',
    'Hogares de 5 o más  personas': 'Households with five or more people'
}

In [22]:
df_datos_indicadores.rename(columns=translations, inplace=True)

In [23]:
df_datos_indicadores.columns

Index(['Census section', 'Total people', 'Percentage of women',
       'Percentage of men', 'Average age', 'Percentage of people under 16',
       'Percentage of people between 16 and 64 years',
       'Percentage of people over 64 years', 'Percentage of foreigners',
       'Percentage of people born abroad',
       'Percentage of people pursuing higher education (ages 16 and older)',
       'Percentage of university students (ages 16 and older)',
       'Percentage of people with higher education (ages 16 and older)',
       'Unemployment rate among the active population',
       'Employment rate among people aged 16 and older',
       'Labor force participation rate among people aged 16 and older',
       'Percentage of disability pensioners among people aged 16 and older',
       'Percentage of retirement pensioners among people aged 16 and older',
       'Percentage of people in other inactive statuses among those aged 16 and older',
       'Percentage of students among people aged

### Final Merges :

In [24]:
# df_sexo_edad_sc
# df_sexo_nacionalidad_sc
# df_renta_media
# df_indicadores_leyenda
# df_datos_indicadores

In [25]:
data_frames_sexo = [df_sexo_edad_sc, df_sexo_nacionalidad_sc]
df_merged_sexo = reduce(lambda left,right: pd.merge(left,right,on=['Census section', 'Gender'],
                                            how='left'), data_frames_sexo)

In [26]:
df_merged_sexo

,Gender,Census section,Age,Population,African,American,Asian,Spanish,European (except Spanish),Rest,Total,Total Foreigner
0,Ambos Sexos,0100101001,De 30 a 39 años,135,49,21,3,1277,35,3,1388,111
1,Ambos Sexos,0100101001,De 40 a 49 años,304,49,21,3,1277,35,3,1388,111
2,Ambos Sexos,0100101001,De 50 a 59 años,247,49,21,3,1277,35,3,1388,111
3,Ambos Sexos,0100101001,De 60 a 69 años,133,49,21,3,1277,35,3,1388,111
4,Ambos Sexos,0100101001,De 70 y más años,119,49,21,3,1277,35,3,1388,111
...,...,...,...,...,...,...,...,...,...,...,...,...
763807,Mujer,5200108015,De 50 a 59 años,197,111,12,0,1033,6,0,1162,129
763808,Mujer,5200108015,De 60 a 69 años,139,111,12,0,1033,6,0,1162,129
763809,Mujer,5200108015,De 70 y más años,100,111,12,0,1033,6,0,1162,129
763810,Mujer,5200108015,Menos de 30 años,407,111,12,0,1033,6,0,1162,129


In [27]:
data_frames_sc = [df_merged_sexo, t_final_renta, df_datos_indicadores]
df_merged_cs= reduce(lambda left,right: pd.merge(left,right,on=['Census section'],
                                            how='left'), data_frames_sc)

In [28]:
df_merged_cs.drop(['Spanish','African', 'American', 'Asian', 'European (except Spanish)', 'Rest', 'Total', 'Total Foreigner'], axis=1, inplace=True)

In [29]:
df_merged_cs.drop(columns=['Gender', 'Age'], inplace=True)

In [30]:
aggregation_functions = {col: 'mean' for col in df_merged_cs.columns if col != 'Population'}
aggregation_functions['Population'] = 'sum'

In [31]:
df_grouped = df_merged_cs.groupby('Census section').agg(aggregation_functions)

In [32]:
df_grouped.drop(['Census section'],inplace=True, axis=1)

In [33]:
df_grouped.reset_index(inplace=True)

### Write the processed data to CSV

In [34]:
df_grouped.to_csv('../../../../data/3_external_data/census/mapping_data/INE_DATA_1.csv', index=False)